In [1]:
import cv2
import os
import torch
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

from ultralytics import YOLO

def display(image):
    plt.figure(figsize = (10, 10))
    plt.xticks([]), plt.yticks([])
    plt.imshow(image)
    plt.show()

def create_dir_if_not_exists(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

file_name = "cat_dog.jpg"
model = YOLO("yolov8m.pt")
cat_and_dog_image = cv2.cvtColor(cv2.imread(file_name), cv2.COLOR_BGR2RGB)

In [3]:
train_csv = pd.read_csv("oidv6-train-annotations-bbox.csv")
validation_csv = pd.read_csv("validation-annotations-bbox.csv")
test_csv = pd.read_csv("test-annotations-bbox.csv")
food_label = '/m/02wbm'
train_dataset = train_csv[(train_csv["LabelName"] == food_label)]
validation_dataset = validation_csv[(validation_csv["LabelName"] == food_label)]
test_dataset = test_csv[(test_csv["LabelName"] == food_label)]

In [5]:
datasets = [
    ("train", train_dataset),
    ("validation", validation_dataset),
    ("test", test_dataset)
]

create_dir_if_not_exists('datasets')
create_dir_if_not_exists('datasets/labels')
create_dir_if_not_exists('datasets/images')

for group, dataset in datasets:
    create_dir_if_not_exists(f'datasets/labels/{group}')
    create_dir_if_not_exists(f'datasets/images/{group}')

    for _, line in dataset.iterrows():
        iid = line['ImageID']

        x_center = (line['XMax'] + line['XMin']) / 2
        y_center = (line['YMax'] + line['YMin']) / 2
        width = line['XMax'] - line['XMin']
        height = line['YMax'] - line['YMin']
        

        with open(f'datasets/labels/{group}/{iid}.txt', 'a') as file:
            file.write(f'0 {x_center} {y_center} {width} {height}\n')

    with open(f'{group}.txt', 'a') as file:
        for iid in dataset['ImageID'].unique():
            file.write(f'{group}/{iid}\n')

In [4]:
import yaml

dataset_yaml_filename = 'yolo.yaml'

dataset_info = {
    'train': 'images/train',
    'val': 'images/validation',
    'nc': 1,
    'names': ['Food']
}

with open(dataset_yaml_filename, 'w') as dataset_file:
    yaml.dump(dataset_info, dataset_file, default_flow_style=False)

In [5]:
model = YOLO("yolov8n.pt")
results = model.train(data='yolo.yaml', epochs=10, device='0')

100%|██████████| 6.23M/6.23M [00:01<00:00, 4.04MB/s]
New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Core(TM) i5-8250U 1.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo_dataset_definition.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, strea

KeyboardInterrupt: 

In [8]:
samples = np.random.choice(test_dataset['ImageID'].unique(), 5)
trained_model = YOLO('runs/detect/train/weights/best.pt')

for sample in samples:
    image = cv2.cvtColor(cv2.imread(f"datasets/images/test/{sample}.jpg"), cv2.COLOR_BGR2RGB)
    results = trained_model(image)
    display(results[0].plot())

FileNotFoundError: [Errno 2] No such file or directory: 'runs/detect/train/weights/best.pt'

In [ ]:
sample_names = np.random.choice(test_dataset['ImageID'].unique(), 5)
trained_model = YOLO('runs/detect/train/weights/best.pt')

for sample in sample_names:
    image = cv2.cvtColor(cv2.imread(f"datasets/images/test/{sample}.jpg"), cv2.COLOR_BGR2RGB)
    results = trained_model(image)
    display(results[0].plot())

    for result in results:
      boxes = result.boxes.xyxy

      for box in boxes:
          x, y, w, h = box.cpu().numpy().astype(int)
          image[y:h, x:w] = cv2.GaussianBlur(image[y:h, x:w], (0, 0), sigmaX=10)

    display(image)